# A notebook for exploring how best to measure the change in Mtb

In [1]:
import napari
from cellpose import models
from macrohet import dataio, tile, visualise, tools
import numpy as np
import os

def view(img):
    return napari.Viewer().add_image(img)

from napari_animation import Animation
from tqdm.auto import tqdm

import btrack
import dask.array as da

from skimage.transform import rescale, resize, downscale_local_mean

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_palette("Spectral")
import cv2
from skimage.io import imsave

### Load all metadata

Both the image metadata and the assay layout metadata

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/DATA/macrohet/macrohet_images/Index.idx.xml'

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [ ]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

# Downscale function

In [15]:
def downscale_rgb_images(images, root_dir='/home/dayn/data/macrohet_temp/macrohet_images_ds/', 
                         segmentation=None, scale_factor=5.04):
    # iterate over frames
    for frame in tqdm(range(0, len(images)), desc='Iterating over frames', total=len(images)):
        
        # create output dir
        output_dir = os.path.join(root_dir, f'{row},{column}')
        os.makedirs(output_dir, exist_ok=True)
        # create RGB filename
        fn = os.path.join(output_dir, f'r0{row}c0{column}t{frame}.png')
        
        if os.path.exists(fn):
            continue
        
        # extract the gfp and rfp channels to apply some vis techn
        gfp = images[frame, 0, ...].compute().compute()
        rfp = images[frame, 1, ...].compute().compute()

        # clip the images so that the contrast is more apparent
        contrast_lim_gfp = np.clip(gfp, 358, 5886)
        contrast_lim_rfp = np.clip(rfp, 480, 1300)
        norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)

        # Create an empty RGB image with the same shape as the input image
        rgb_image = np.zeros((norm_gfp.shape[0], norm_gfp.shape[1], 3), dtype=np.uint16)

        # Assign the first channel to the green channel of the RGB image
        rgb_image[:, :, 1] = norm_gfp

        # Assign the second channel to the red and blue channels of the RGB image to create magenta
        rgb_image[:, :, 0] = norm_rfp
        rgb_image[:, :, 2] = norm_rfp

        # scale down to 8-bit
        rgb_image = np.uint8(rgb_image >> 8)

        # create mask outline
        if segmentation is not None:
            # get relevant frame
            masks = segmentation[frame]
            # iterate over each individual segment, drawing it onto the original RGB image
            for seg_ID in tqdm(range(1, np.max(masks)), desc='Creating outlines of masks', leave=False):
                instance_mask = (masks == seg_ID).astype(np.uint8)
                # draw outline
                contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(rgb_image, contours, -1, (0, 2**8, 2**8), thickness=2)  # make 8-bit

        # downsize image to reduce storage demands
        rgb_image = cv2.resize(rgb_image, (int(rgb_image.shape[1] // scale_factor), int(rgb_image.shape[0] // scale_factor)))

        # save out RGB image
        imsave(fn, rgb_image)

    print(f'Position {row}, {column} complete')


In [27]:
for position, key in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
        
    row = position[0]
    column = position[1]

    # pre-load images from set time point
    base_dir = '/mnt/DATA/macrohet/'
    image_dir = os.path.join(base_dir, 'macrohet_images/Images')
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, column, 
                                 set_plane='sum_proj',
                                 )
    ### load segmentation
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, 
                                                f'labels/macrohet_seg_model/{int(row),int(column)}.h5'), 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
        segmentation = reader.segmentation
    
    # downscale
    downscale_rgb_images(images, 
                         root_dir = '/home/dayn/data/macrohet_temp/macrohet_images_ds_2x/', 
                         segmentation = segmentation, scale_factor=2.52)


  0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/07/13 02:42:47 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/07/13 02:42:57 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/07/13 02:42:57 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5


Iterating over frames:   0%|          | 0/75 [00:00<?, ?it/s]

Creating outlines of masks:   0%|          | 0/641 [00:00<?, ?it/s]

Creating outlines of masks:   0%|          | 0/629 [00:00<?, ?it/s]

...

Creating outlines of masks:   0%|          | 0/262 [00:00<?, ?it/s]

Creating outlines of masks:   0%|          | 0/258 [00:00<?, ?it/s]

Creating outlines of masks:   0%|          | 0/245 [00:00<?, ?it/s]

Creating outlines of masks:   0%|          | 0/251 [00:00<?, ?it/s]

Position 6, 9 complete


In [18]:
from macrohet import notify

In [28]:
notify.send_sms('all positions done for 2x')